In [5]:
!pip install gym
!pip install stable_baselines3

In [6]:
from gym import Env
from gym.spaces import Discrete, Box
import random
import numpy as np
import torch
from stable_baselines3 import SAC, PPO
import gym

In [7]:
t = np.array([0.0, 0.0])
type(t[0])

numpy.float64

In [15]:
### Define a two line manifold environment ###
class simEnv(Env):
    def __init__(self):
        # Action agent can take
        self.action_space = Box(np.array([-1, -1]), np.array([1, 1]))
        # Work space
        self.observation_space = Box(np.array([-10, -10]), np.array([10, 10]))
        # Initial starting point
        self.state = np.array([0.0, 0.0])
        # Goal state
        self.goal = np.array([10.0, 10.0])
        # Intersection point
        self.intersection = np.array([10.0, 0.0])
        # Indicator whether next manifold is reached or not
        self.next_manifold = False
    
    def step(self, action):
        ### Projection ###
        if self.next_manifold:
            _action = np.array([0.0, action[1]])
        else:
            _action = np.array([action[0], 0.0])
        self.state += _action
        if self.state[0] == 10:
            self.next_manifold = True
        done = False
        # Design rewards
        if self.state[1] != 0 and self.state[0] != 10: # Fall off the manifold
            # negative of l2 distance to current manifold reward
            if not self.next_manifold:
                reward = -np.abs(self.state[1])
            else:
                reward = -np.abs(self.state[0] - 10)
            reward = -100
            done = True
        elif np.array_equal(self.state, self.intersection): # Get to the intersection point
            if self.next_manifold:
                reward = 1 / np.linalg.norm(self.state - self.goal)
            else:
                reward = 10
        elif np.array_equal(self.state, self.goal): # Reach the goal 
            reward = 100
            done = True
        else: # Direct the agent to intersection point / goal
            if self.next_manifold:
                reward = -np.linalg.norm(self.state - self.goal)
#                 reward = 1 / np.linalg.norm(self.state - self.goal)
            else: 
                reward = -np.linalg.norm(self.state - self.intersection)
#                 reward = 1 / np.linalg.norm(self.state - self.intersection)
        # Set placeholder for info 
        info = {}
        return self.state, reward, done, info
    
    def render(self):
        pass
    
    def reset(self):
        self.state = np.array([0.0,0.0])
        self.next_manifold = False
        return self.state

In [16]:
env = simEnv()

In [17]:
device = torch.device(
        "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = PPO('MlpPolicy', env, verbose=1, device=device)
model = model.learn(total_timesteps=500000, eval_freq=1000)

Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 1531 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1130         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0068806177 |
|    clip_fraction        | 0.0671       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.85        |
|    explained_variance   | -0.00666     |
|    learning_rate        | 0.0003       |
|    loss                 | 8.83e+04     |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00439     |
|    

-----------------------------------------
| time/                   |             |
|    fps                  | 833         |
|    iterations           | 12          |
|    time_elapsed         | 29          |
|    total_timesteps      | 24576       |
| train/                  |             |
|    approx_kl            | 0.020257652 |
|    clip_fraction        | 0.178       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.54       |
|    explained_variance   | 2.93e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 1.01        |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.0217     |
|    std                  | 0.862       |
|    value_loss           | 2.39        |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 823         |
|    iterations           | 13          |
|    time_elapsed         | 32    

-----------------------------------------
| time/                   |             |
|    fps                  | 803         |
|    iterations           | 23          |
|    time_elapsed         | 58          |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.014747739 |
|    clip_fraction        | 0.165       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.87       |
|    explained_variance   | -0.00922    |
|    learning_rate        | 0.0003      |
|    loss                 | 0.152       |
|    n_updates            | 220         |
|    policy_gradient_loss | 0.00424     |
|    std                  | 0.674       |
|    value_loss           | 0.381       |
-----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 802        |
|    iterations           | 24         |
|    time_elapsed         | 61        

----------------------------------------
| time/                   |            |
|    fps                  | 808        |
|    iterations           | 34         |
|    time_elapsed         | 86         |
|    total_timesteps      | 69632      |
| train/                  |            |
|    approx_kl            | 0.01502686 |
|    clip_fraction        | 0.23       |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.33      |
|    explained_variance   | -0.00252   |
|    learning_rate        | 0.0003     |
|    loss                 | 0.114      |
|    n_updates            | 330        |
|    policy_gradient_loss | -0.00534   |
|    std                  | 0.589      |
|    value_loss           | 0.273      |
----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 809         |
|    iterations           | 35          |
|    time_elapsed         | 88          |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 813          |
|    iterations           | 45           |
|    time_elapsed         | 113          |
|    total_timesteps      | 92160        |
| train/                  |              |
|    approx_kl            | 0.0146023035 |
|    clip_fraction        | 0.27         |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.799       |
|    explained_variance   | -0.00409     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.0251       |
|    n_updates            | 440          |
|    policy_gradient_loss | 0.0182       |
|    std                  | 0.53         |
|    value_loss           | 0.0843       |
------------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 813         |
|    iterations           | 46          |
|    time_elaps

...

----------------------------------------
| time/                   |            |
|    fps                  | 852        |
|    iterations           | 199        |
|    time_elapsed         | 478        |
|    total_timesteps      | 407552     |
| train/                  |            |
|    approx_kl            | 0.07308593 |
|    clip_fraction        | 0.412      |
|    clip_range           | 0.2        |
|    entropy_loss         | 0.878      |
|    explained_variance   | 1.29e-05   |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0212    |
|    n_updates            | 1980       |
|    policy_gradient_loss | 0.0128     |
|    std                  | 0.524      |
|    value_loss           | 0.243      |
----------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 852        |
|    iterations           | 200        |
|    time_elapsed         | 480        |
|    total_times

---------------------------------------
| time/                   |           |
|    fps                  | 854       |
|    iterations           | 210       |
|    time_elapsed         | 503       |
|    total_timesteps      | 430080    |
| train/                  |           |
|    approx_kl            | 0.0759561 |
|    clip_fraction        | 0.361     |
|    clip_range           | 0.2       |
|    entropy_loss         | 0.834     |
|    explained_variance   | 0.0136    |
|    learning_rate        | 0.0003    |
|    loss                 | 0.00987   |
|    n_updates            | 2090      |
|    policy_gradient_loss | 0.000393  |
|    std                  | 0.528     |
|    value_loss           | 0.123     |
---------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 854        |
|    iterations           | 211        |
|    time_elapsed         | 505        |
|    total_timesteps      | 432128 

-----------------------------------------
| time/                   |             |
|    fps                  | 856         |
|    iterations           | 221         |
|    time_elapsed         | 528         |
|    total_timesteps      | 452608      |
| train/                  |             |
|    approx_kl            | 0.061620496 |
|    clip_fraction        | 0.367       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.817       |
|    explained_variance   | 0.0701      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.112       |
|    n_updates            | 2200        |
|    policy_gradient_loss | -0.00735    |
|    std                  | 0.536       |
|    value_loss           | 0.183       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 856         |
|    iterations           | 222         |
|    time_elapsed         | 531   

-----------------------------------------
| time/                   |             |
|    fps                  | 857         |
|    iterations           | 232         |
|    time_elapsed         | 554         |
|    total_timesteps      | 475136      |
| train/                  |             |
|    approx_kl            | 0.042024303 |
|    clip_fraction        | 0.363       |
|    clip_range           | 0.2         |
|    entropy_loss         | 0.713       |
|    explained_variance   | 0.0626      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0503      |
|    n_updates            | 2310        |
|    policy_gradient_loss | 0.00465     |
|    std                  | 0.581       |
|    value_loss           | 0.146       |
-----------------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 857         |
|    iterations           | 233         |
|    time_elapsed         | 556   

--------------------------------------
| time/                   |          |
|    fps                  | 858      |
|    iterations           | 243      |
|    time_elapsed         | 579      |
|    total_timesteps      | 497664   |
| train/                  |          |
|    approx_kl            | 0.034869 |
|    clip_fraction        | 0.346    |
|    clip_range           | 0.2      |
|    entropy_loss         | 0.821    |
|    explained_variance   | 0.0582   |
|    learning_rate        | 0.0003   |
|    loss                 | 0.0112   |
|    n_updates            | 2420     |
|    policy_gradient_loss | -0.0153  |
|    std                  | 0.546    |
|    value_loss           | 0.11     |
--------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 859        |
|    iterations           | 244        |
|    time_elapsed         | 581        |
|    total_timesteps      | 499712     |
| train/     

In [18]:
obs = env.reset()

for _ in range(100):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    print('=====')
    print(obs)
    print(rewards)

=====
[1. 0.]
-9.0
=====
[2. 0.]
-8.0
=====
[3. 0.]
-7.0
=====
[4. 0.]
-6.0
=====
[5. 0.]
-5.0
=====
[6. 0.]
-4.0
=====
[7. 0.]
-3.0
=====
[7.78907472 0.        ]
-2.210925281047821
=====
[8.37890512 0.        ]
-1.6210948824882507
=====
[8.78755832 0.        ]
-1.2124416828155518
=====
[9.10008737 0.        ]
-0.8999126255512238
=====
[9.32068688 0.        ]
-0.6793131232261658
=====
[9.55418406 0.        ]
-0.44581593573093414
=====
[9.68067454 0.        ]
-0.3193254619836807
=====
[9.75936401 0.        ]
-0.24063599109649658
=====
[9.87143537 0.        ]
-0.12856463342905045
=====
[9.94522381 0.        ]
-0.05477619171142578
=====
[10.02678749  0.        ]
-0.026787489652633667
=====
[10.11129017  0.        ]
-0.11129017174243927
=====
[10.12255942  0.        ]
-0.12255942448973656
=====
[10.1243435  0.       ]
-0.12434350326657295
=====
[10.12768865  0.        ]
-0.12768864631652832
=====
[10.17239237  0.        ]
-0.1723923720419407
=====
[10.1929385  0.       ]
-0.192938502877950

In [ ]:
# env = simEnv()

# device = torch.device(
#         "cuda:0") if torch.cuda.is_available() else torch.device("cpu")
# model = SAC('MlpPolicy', env, verbose=1, device=device)
# model = model.learn(total_timesteps=500000, eval_freq=1000)

Using cuda:0 device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
# obs = env.reset()

# for _ in range(100):
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)
#     print(obs)
#     print(rewards)